In [87]:
import glob
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.

csv_path = '../price_predict/vegetable_price/original_vegetable_csv/'
vegetable_names = ['청양', '시금치(일반)', '토마토(일반)', '파프리카(일반)', '깻잎(일반)', '미나리(일반)', '양파(일반)', '풋고추(전체)']
csv_files = glob.glob(csv_path + '*.csv')

In [116]:
def create_today_df():
    today = datetime.today().strftime('%Y-%m-%d')
    # 날짜 범위를 데이터프레임에 추가합니다.
    date_range = pd.date_range(start='2023-01-01', end=today)
    return pd.DataFrame(date_range, columns=['일자'])

In [117]:
class VegetablePriceReader:
    # csv_file를 받아서 초기화하는 생성자 함수
    def __init__(self, csv_file):
        self.vege_name = os.path.basename(csv_file.split()[0])
        self.df = pd.read_csv(csv_file)
        self.df = self.df.drop(self.df.columns[1], axis=1).fillna(0)

    # '일자' 열을 확인하고 오늘 날짜 이전의 모든 행을 필터링하는 함수
    def filter_rows(self):
        self.df['일자'] = pd.to_datetime(self.df['일자'].astype(str), format='%Y%m%d')
        today = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
        self.df = self.df[self.df['일자'] <= today]

    # 위의 네 가지 함수를 순서대로 실행하는 함수
    def process(self):
        col_name = self.df.columns[1]
        self.df = self.df.rename(columns={col_name: self.vege_name + " " + col_name})
        self.filter_rows()

        all_dates = create_today_df()
        # 원래 데이터프레임과 all_dates를 병합
        df = pd.merge(all_dates, self.df, on='일자', how='left')
        df['일자'] = df['일자'].dt.strftime('%Y-%m-%d')
        return df.iloc[:,1]




In [118]:
df = create_today_df()
for csv_path in csv_files:
    reader = VegetablePriceReader(csv_path)
    new_df = reader.process()
    df = pd.concat([df, new_df], axis=1)

df['일자'] = df['일자'].astype(str)
save = df.set_index('일자').to_json('output.json', orient='index')

In [121]:
with open('output.json', 'r') as f:
    data = json.load(f)
print(data)

{'2023-01-01': {'청양 거래물량(톤)': 0.0, '시금치(일반) 거래물량(톤)': 0.0, '깻잎(일반) 평균가격(원/kg)': 0.0, '파프리카(일반) 평균가격(원/kg)': 0.0, '풋고추(전체) 거래물량(톤)': 0.0, '양파(일반) 평균가격(원/kg)': 0.0, '파프리카(일반) 거래물량(톤)': 0.0, '미나리(일반) 평균가격(원/kg)': 0.0, '청양 평균가격(원/kg)': 0.0, '양파(일반) 거래물량(톤)': 0.0, '풋고추(전체) 평균가격(원/kg)': 0.0, '토마토(일반) 거래물량(톤)': 0.0, '미나리(일반) 거래물량(톤)': 0.0, '시금치(일반) 평균가격(원/kg)': 0.0, '깻잎(일반) 거래물량(톤)': 0.0, '토마토(일반) 평균가격(원/kg)': 0.0}, '2023-01-02': {'청양 거래물량(톤)': 26.51, '시금치(일반) 거래물량(톤)': 45.63, '깻잎(일반) 평균가격(원/kg)': 20400.0, '파프리카(일반) 평균가격(원/kg)': 4650.0, '풋고추(전체) 거래물량(톤)': 33.31, '양파(일반) 평균가격(원/kg)': 1310.0, '파프리카(일반) 거래물량(톤)': 0.74, '미나리(일반) 평균가격(원/kg)': 3590.0, '청양 평균가격(원/kg)': 7170.0, '양파(일반) 거래물량(톤)': 29.41, '풋고추(전체) 평균가격(원/kg)': 6600.0, '토마토(일반) 거래물량(톤)': 16.32, '미나리(일반) 거래물량(톤)': 8.5, '시금치(일반) 평균가격(원/kg)': 1840.0, '깻잎(일반) 거래물량(톤)': 3.89, '토마토(일반) 평균가격(원/kg)': 1620.0}, '2023-01-03': {'청양 거래물량(톤)': 125.5, '시금치(일반) 거래물량(톤)': 148.99, '깻잎(일반) 평균가격(원/kg)': 18450.0, '파프리카(일반) 평균가격(원/kg)': 4270.0, '풋고추(전체) 거래물량(